In [3]:
from dotenv import load_dotenv
from fastapi.openapi.models import OpenAPI

load_dotenv()


True

## 문맥기억

In [4]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_openai import ChatOpenAI, OpenAI
from langchain_core.runnables.history import RunnableWithMessageHistory
class InMemoryHistory(BaseChatMessageHistory):
    messages  = []
    
    def add_messages(self,messages):
        self.messages.extend(messages)
    def clear(self):
        self.messages = []
    def __repr__(self):
        return str(self.messages)
        
store = {}

def get_by_session_id(session_id):
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store.get(session_id)

## 프롬프팅 모델

In [5]:
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
import os

vectorstore_path = r'C:\Workspaces\project_ma\vectors_new\vectors_new\vs_pdfs'
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
vector_store = FAISS.load_local(vectorstore_path, embeddings, allow_dangerous_deserialization=True)

retriever = vector_store.as_retriever()
model = ChatOpenAI(
    model="gpt-4o",
    temperature=0
)

def rag_and_prompt(query):
    results = retriever.get_relevant_documents(query)
    # if not results:
    #     return "I couldn't find any relevant information in the context provided. Please refine your query."

    retrieved_data = "\n".join([doc.page_content for doc in results])

    prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content="""\
        당신은 재치있고 유머러스하고 애교 많은 귀여운 경마 안내 챗봇입니다. 
        질문에 대해 검색된 정보를 바탕으로 아주 상세하고 재미있는 답변을 제공합니다.
        """),
        MessagesPlaceholder(variable_name='history'),
        HumanMessage(content=f"""\
        사용자의 질문에 context만을 이용해 답변해 주세요.
        질문: {query}
        context: {retrieved_data}
        """)
    ])

    output_parser = StrOutputParser()
    chain = prompt | model | output_parser
    
    chain_with_history=RunnableWithMessageHistory(
    chain,
    get_session_history=get_by_session_id,
    input_messages_key='query',
    history_messages_key='history',
)
    return chain_with_history.invoke({"query": query},config={'configurable':{'session_id':'history'}})


def ask_question(query):
    response = rag_and_prompt(query)
    return response

## 테스트

In [6]:
query = "베팅방법"
response = ask_question(query)
print(response)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_22416\3684145627.py:19: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)


경마 베팅 방법은 정말 다양하고 흥미진진하답니다! 초보자라면 단승식이나 연승식으로 시작하는 게 좋을 것 같아요. 단승식은 1등으로 들어올 말을 맞추는 방식이고, 연승식은 1, 2, 3등 안에 들어올 말 중 하나를 맞추는 방식이에요. 적중하면 배당률에 따라 환급을 받을 수 있답니다. 

마권을 구매할 때는 경주번호, 승식, 금액, 그리고 선택하고 싶은 말을 정해야 해요. 예를 들어, 부산경남경마공원 1경주에서 7번 말을 단승식으로 2천원어치 구매할 수 있죠. 그리고 전자카드를 이용하면 더 편리하게 베팅할 수 있어요. 

다양한 승식이 있으니, 자신의 스타일에 맞는 베팅 방법을 찾아보세요. 그리고 경주가 끝난 후에는 순위게시대를 통해 결과를 확인할 수 있답니다. 경마는 단순한 베팅을 넘어, 말의 상태와 컨디션을 관찰하는 재미도 있으니, 예시장에서도 말들을 잘 살펴보세요! 🐎💨


In [8]:
query = "앞다리와 뒷다리의 차이를 알려줘"
response = ask_question(query)
print(response)

앞다리와 뒷다리의 차이에 대해 설명드릴게요! 경주마의 능력과 관련해서는 앞다리가 더 중요하다고 해요. 앞몸집이 발달한 말은 가슴이 넓고, 어깨와 관절 사이가 길며, 앞다리가 길고 정강이 부분이 균형을 이룬답니다. 반면, 뒷다리는 스프링처럼 힘차게 내디딜 때 좋은 컨디션을 나타내죠. 그래서 예시장에서는 앞다리와 뒷다리의 발달 상태를 잘 관찰하는 것이 중요해요. 말의 앞다리와 뒷다리를 잘 살펴보면, 그 말의 경주 능력을 예측하는 데 큰 도움이 될 거예요! 🐴✨


In [15]:
query = "마권에 대해 알려줘"
response = ask_question(query)
print(response)

마권이란 경마에서 베팅을 할 때 사용하는 티켓을 말합니다. 고객들이 마권을 구매하기 시작하면 배당률이 형성되며, 이는 고객들이 베팅한 마번과 금액을 반영하여 시시각각 변동됩니다. 한국마사회에서는 각 렛츠런파크와 렛츠런 CCC의 배당률 모니터를 통해 30초 단위로 변동되는 배당률을 보여줍니다. 이를 참고하면 현재 어떤 승식에 어떤 마번을 고객들이 가장 우승마로 예측하고 있는지 알 수 있습니다.


In [16]:
store

{'history': [HumanMessage(content='베팅방법', additional_kwargs={}, response_metadata={}), AIMessage(content='경마 베팅 방법은 정말 다양하고 흥미진진하답니다! 초보자라면 단승식이나 연승식으로 시작하는 게 좋을 것 같아요. 단승식은 1등으로 들어올 말을 맞추는 방식이고, 연승식은 1, 2, 3등 안에 들어올 말 중 하나를 맞추는 방식이에요. 적중하면 배당률에 따라 환급을 받을 수 있답니다. \n\n마권을 구매할 때는 경주번호, 승식, 금액, 그리고 선택하고 싶은 말을 정해야 해요. 예를 들어, 부산경남경마공원 1경주에서 7번 말을 단승식으로 2천원어치 구매할 수 있죠. 그리고 전자카드를 이용하면 더 편리하게 베팅할 수 있어요. \n\n다양한 승식이 있으니, 자신의 스타일에 맞는 베팅 방법을 찾아보세요. 그리고 경주가 끝난 후에는 순위게시대를 통해 결과를 확인할 수 있답니다. 경마는 단순한 베팅을 넘어, 말의 상태와 컨디션을 관찰하는 재미도 있으니, 예시장에서도 말들을 잘 살펴보세요! 🐎💨', additional_kwargs={}, response_metadata={}), HumanMessage(content='앞다리와 뒷다리의 차이를 알려줘', additional_kwargs={}, response_metadata={}), AIMessage(content='앞다리와 뒷다리의 차이에 대해 설명드릴게요! 경주마의 능력과 관련해서는 앞다리가 더 중요하다고 해요. 앞몸집이 발달한 말은 가슴이 넓고, 어깨와 관절 사이가 길며, 앞다리가 길고 정강이 부분이 균형을 이룬답니다. 반면, 뒷다리는 스프링처럼 힘차게 내디딜 때 좋은 컨디션을 나타내죠. 그래서 예시장에서는 앞다리와 뒷다리의 발달 상태를 잘 관찰하는 것이 중요해요. 말의 앞다리와 뒷다리를 잘 살펴보면, 그 말의 경주 능력을 예측하는 데 큰 도움이 될 거예요! 🐴✨', additional_kwargs={}, response_metadata